## Preparation des donnees et features ingeneering

In [24]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler
import joblib
import sys
sys.path.append('/app/notebooks/openstack')
from utils import *

In [25]:
## Charger données prétraitées
df = load_data()
event_cols = get_event_columns(df)

print(f"Shape: {df.shape}")
print(df.head())
print(f"Labels: {df['Label'].value_counts()}")

Shape: (2069, 26)
                             InstanceId   Label  E1  E2  E3  E4  E5  E6  E7  \
0  00287154-fceb-41a6-827d-713086dd80c7  Normal   1   3   6   0   1   6   1   
1  002afded-8d0b-49f2-bab7-07e8ff79eac4  Normal   0   6   4   0   0   3   0   
2  0039c90c-b94c-4a97-985c-0421663a49fc  Normal   1   3   6   0   1   6   1   
3  00533c6c-c5fd-4910-ad06-508ea82a2c2e  Normal   1   3   6   0   1   6   1   
4  0060e3e6-bfb7-4eaf-937b-30c303472315  Normal   1   3   6   0   1   6   1   

   E8  ...  E15  E16  E17  E18  E19  E20  E21  E22  E23  TotalEvents  
0   1  ...    0    0    0    0    0    0    0    0    0           25  
1   2  ...    6    1    0    0    0    0    0    0    0           25  
2   1  ...    0    0    0    0    0    0    0    0    0           24  
3   1  ...    0    0    0    0    0    0    0    0    0           25  
4   1  ...    0    0    0    0    0    0    0    0    0           24  

[5 rows x 26 columns]
Labels: Label
Normal     1871
Anomaly     198
Name: count,

In [26]:
## Preparation des variables X et y

#Features : toutes les colonnes d'événements
X = df[event_cols].fillna(0)

#Target : Anomaly (1) ou Normal (0)
y = (df['Label'] == 'Anomaly').astype(int)

print(f"X: {X.shape}")
print(f"y: Normal={sum(y==0):,}, Anomaly={sum(y==1):,}")
print(f"Ratio: {sum(y==0)/sum(y==1):.1f}:1")

X: (2069, 23)
y: Normal=1,871, Anomaly=198
Ratio: 9.4:1


In [27]:
# # Suppression des features a faibles variance qui n'apportent pas d'information
# variance = X.var().sort_values()
# print(f"\nVariance des evenements:")
# print(variance.head(10))

# low_variance = variance[variance < 0.01]
# print(f"\nevenements avec variance < 0.01: {len(low_variance)}")

# to_drop_variance = low_variance.index.tolist()
# X = X.drop(columns=to_drop_variance)

# print(f"Features restantes: {X.shape[1]}")

In [28]:
# # Suppression correlations elevees
# corr_matrix = X.corr().abs()
# upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(bool))
# to_drop_corr = [col for col in upper.columns if any(upper[col] > 0.95)]

# X = X.drop(columns=to_drop_corr)
# print(f"Features apres suppression correlations elevees: {X.shape[1]}")

# # Visualisation
# plt.figure(figsize=(15, 12))
# sns.heatmap(
#     X.corr(), 
#     annot=False, 
#     cmap='viridis',
#     linewidths=.5,
#     center=0
# )

### Sauvagardes des donnees
On met toutes les donnees sur une meme echelle afin d'ameliorer la performance du modele. (E5: 1723232 vers E24: 4)
- X_scaled : Features normalisees
- y : Labels (0=Success, 1=Fail)
- scaler : Pour transformer de nouvelles données


In [29]:
joblib.dump(X, PROCESSED_DIR / 'X_preprocessed.pkl')
joblib.dump(y, PROCESSED_DIR / 'y_preprocessed.pkl')

print("Donnees pretraitees sauvegardees.")
print(f" {X.shape[0]} echantillons")

Donnees pretraitees sauvegardees.
 2069 echantillons
